### MID 27
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### CNN model used, model type is 2 as per the BTP documentation spreadsheet.
### PCA with n_components = 10
### np.rint() used on predictions

In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Dense,Dropout,Conv2D,Conv1D,Flatten,MaxPool2D
import tensorflow as tf
import csv
from sklearn.decomposition import PCA

In [12]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [13]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

In [14]:
combined_data

name  version                                             name.1  \
0    ant      1.3            org.apache.tools.ant.taskdefs.ExecuteOn   
1    ant      1.3                 org.apache.tools.ant.DefaultLogger   
2    ant      1.3     org.apache.tools.ant.taskdefs.TaskOutputStream   
3    ant      1.3                  org.apache.tools.ant.taskdefs.Cvs   
4    ant      1.3             org.apache.tools.ant.taskdefs.Copyfile   
..   ...      ...                                                ...   
173  ant      1.4                   org.apache.tools.ant.TaskAdapter   
174  ant      1.4  org.apache.tools.ant.taskdefs.rmic.DefaultRmic...   
175  ant      1.4           org.apache.tools.ant.IntrospectionHelper   
176  ant      1.4  org.apache.tools.ant.taskdefs.compilers.Defaul...   
177  ant      1.4                org.apache.tools.ant.NoBannerLogger   

          wmc  dit    noc       cbo       rfc      lcom        ca  ...  \
0    0.142857  0.6  0.050  0.102941  0.214286  0.011856  0.014815  ...   
1    0.181818  0.0  0.025  0.058824  0.163265  0.020033  0.029630  ...   
2    0.038961  0.2  0.000  0.007353  0.045918  0.000000  0.000000  ...   
3    0.155844  0.4  0.000  0.088235  0.188776  0.013083  0.000000  ...   
4    0.077922  0.4  0.000  0.029412  0.107143  0.000409  0.000000  ...   
..        ...  ...    ...       ...       ...       ...       ...  ...   
173  0.064935  0.4  0.000  0.051471  0.096939  0.001635  0.029630  ...   
174  0.220779  0.0  0.075  0.110294  0.336735  0.036795  0.029630  ...   
175  0.298701  0.0  0.000  0.227941  0.418367  0.082175  0.162963  ...   
176  0.142857  0.0  0.200  0.161765  0.438776  0.006132  0.059259  ...   
177  0.051948  0.2  0.000  0.022059  0.081633  0.000000  0.000000  ...   

          dam       moa       mfa       cam    ic       cbm       amc  \
0    1.000000  0.111111  0.885057  0.232323  0.75  0.363636  0.165951   
1    1.000000  0.000000  0.000000  0.307692  0.00  0.000000  0.080979   
2    1.000000  0.111111  0.714286  0.666667  0.25  0.090909  0.083267   
3    1.000000  0.111111  0.770833  0.458333  0.00  0.000000  0.115693   
4    1.000000  0.000000  0.880952  0.416667  0.50  0.181818  0.100881   
..        ...       ...       ...       ...   ...       ...       ...   
173  0.000000  0.000000  0.902439  0.400000  0.25  0.090909  0.138351   
174  1.000000  0.222222  0.000000  0.197917  0.00  0.000000  0.172373   
175  0.444444  0.000000  0.000000  0.318182  0.00  0.000000  0.219724   
176  1.000000  0.777778  0.000000  0.266667  0.00  0.000000  0.373390   
177  1.000000  0.000000  0.842105  0.875000  0.25  0.090909  0.064852   

       max_cc    avg_cc  bug  
0    0.085714  0.209085    0  
1    0.171429  0.269903    2  
2    0.028571  0.109529    0  
3    0.085714  0.232742    0  
4    0.028571  0.136898    0  
..        ...       ...  ...  
173  0.028571  0.131428    0  
174  0.400000  0.396221    0  
175  0.685714  0.378561    0  
176  0.628571  0.761689    1  
177  0.142857  0.287498    0  

[303 rows x 24 columns]

In [15]:
# define transform
components = 10
pca = PCA(n_components=components)
# prepare transform on dataset
pca.fit(combined_data[cols_to_norm])
# apply transform to dataset
transformed = pca.transform(combined_data[cols_to_norm])

In [16]:
transformed = pd.DataFrame(transformed)
transformed 

0         1         2         3         4         5         6  \
0   -0.818904  0.255541  0.173010 -0.085265 -0.298426 -0.041536 -0.029952   
1    0.211850 -0.596714 -0.058649 -0.060459 -0.059547 -0.063873 -0.088019   
2   -0.310718 -0.021222 -0.538803 -0.015225 -0.015308  0.025091  0.105172   
3   -0.328366  0.034184 -0.105838 -0.033729  0.314725 -0.066018 -0.185431   
4   -0.516435  0.234332 -0.202375 -0.122822 -0.155654 -0.028101 -0.014558   
..        ...       ...       ...       ...       ...       ...       ...   
298  0.126544  0.701796  0.109947 -0.104515  0.057198 -0.305038  0.278453   
299  0.071521 -0.725060  0.254031  0.156662  0.066015 -0.159664 -0.159516   
300  0.276368 -0.485600  0.763979  0.403732  0.309357 -0.081984 -0.039603   
301 -0.023641 -0.851754  0.470180  0.584599  0.163868 -0.184260 -0.144292   
302 -0.305827  0.130311 -0.516061  0.225862  0.071176  0.149595  0.066592   

            7         8         9  
0    0.060052 -0.061844  0.129277  
1    0.048838  0.136331 -0.023472  
2   -0.187430 -0.136162 -0.004927  
3    0.118595 -0.020596 -0.020034  
4   -0.023515  0.081734  0.039900  
..        ...       ...       ...  
298 -0.261710 -0.178725 -0.005567  
299  0.152804 -0.060837  0.065179  
300 -0.037146 -0.153829  0.234048  
301  0.435575 -0.433720  0.035253  
302 -0.080964 -0.008366 -0.034391  

[303 rows x 10 columns]

In [17]:
train_data_index_list = list(range(ant_1_3.shape[0]))
train_data_index_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124]

In [18]:
test_data_index_list = list(range(ant_1_3.shape[0], ant_1_3.shape[0] + ant_1_4.shape[0]))
test_data_index_list

[125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291

In [19]:
X_train = transformed[transformed.index.isin(train_data_index_list)]
X_train = np.array(X_train)
Y_train = combined_data[transformed.index.isin(train_data_index_list)]
Y_train = Y_train['bug']

X_test = transformed[transformed.index.isin(test_data_index_list)]
X_test = np.array(X_test)
Y_test = combined_data[transformed.index.isin(test_data_index_list)]
Y_test = Y_test['bug']

In [20]:
# Applying over sampling and SMOTE to training data for augmentation.
'''
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)
'''

'\nros = RandomOverSampler(random_state=0)\nX_train, Y_train = ros.fit_resample(X_train, Y_train)\n\nsmt = SMOTE()\nX_train, Y_train = smt.fit_resample(X_train, Y_train)\n'

In [21]:
#Creating X_train dataframe after SMOTE
X_train = pd.DataFrame(X_train)
X_train

0         1         2         3         4         5         6  \
0   -0.818904  0.255541  0.173010 -0.085265 -0.298426 -0.041536 -0.029952   
1    0.211850 -0.596714 -0.058649 -0.060459 -0.059547 -0.063873 -0.088019   
2   -0.310718 -0.021222 -0.538803 -0.015225 -0.015308  0.025091  0.105172   
3   -0.328366  0.034184 -0.105838 -0.033729  0.314725 -0.066018 -0.185431   
4   -0.516435  0.234332 -0.202375 -0.122822 -0.155654 -0.028101 -0.014558   
..        ...       ...       ...       ...       ...       ...       ...   
120 -0.807173 -0.481510  1.292529 -0.038640  0.182542  0.633477 -0.282391   
121 -0.486862 -0.093769 -0.036945 -0.417515  0.165363  0.188567  0.756006   
122  0.356342 -0.437324  0.555356  0.461422  0.171109 -0.193366  0.052797   
123 -0.305827  0.130311 -0.516061  0.225862  0.071176  0.149595  0.066592   
124  0.705844 -0.171467  0.465241  0.062213  0.053179 -0.146501  0.185184   

            7         8         9  
0    0.060052 -0.061844  0.129277  
1    0.048838  0.136331 -0.023472  
2   -0.187430 -0.136162 -0.004927  
3    0.118595 -0.020596 -0.020034  
4   -0.023515  0.081734  0.039900  
..        ...       ...       ...  
120 -0.302450 -0.112322 -0.210326  
121  0.317682  0.054703  0.104190  
122  0.015626 -0.121095  0.182815  
123 -0.080964 -0.008366 -0.034391  
124 -0.332544 -0.211090 -0.183971  

[125 rows x 10 columns]

In [22]:
#Creating Y_train dataframe after SMOTE
Y_train = pd.DataFrame(Y_train, columns=['bug'])
Y_train

bug
0      0
1      2
2      0
3      0
4      0
..   ...
120    0
121    0
122    2
123    0
124    0

[125 rows x 1 columns]

In [23]:
#Creating X_test dataframe after SMOTE
X_test = pd.DataFrame(X_test)
X_test 

0         1         2         3         4         5         6  \
0   -0.435557  0.006613 -0.001504 -0.037302  0.084933 -0.075094 -0.110340   
1   -0.585951  0.259560 -0.139178  0.010484 -0.100906  0.056142 -0.061287   
2    0.276324 -0.506007 -0.322392 -0.101244 -0.056399  0.169781 -0.051785   
3   -0.734317  0.341617 -0.030143  0.038302 -0.285238  0.064460  0.013935   
4    0.004661 -0.674220  0.266761 -0.169607 -0.127306  0.058459 -0.065243   
..        ...       ...       ...       ...       ...       ...       ...   
173  0.126544  0.701796  0.109947 -0.104515  0.057198 -0.305038  0.278453   
174  0.071521 -0.725060  0.254031  0.156662  0.066015 -0.159664 -0.159516   
175  0.276368 -0.485600  0.763979  0.403732  0.309357 -0.081984 -0.039603   
176 -0.023641 -0.851754  0.470180  0.584599  0.163868 -0.184260 -0.144292   
177 -0.305827  0.130311 -0.516061  0.225862  0.071176  0.149595  0.066592   

            7         8         9  
0    0.081049 -0.045027 -0.046944  
1   -0.063591  0.006644  0.213866  
2   -0.050898  0.055162 -0.007508  
3   -0.005841  0.062722  0.031463  
4   -0.217234  0.254638 -0.040460  
..        ...       ...       ...  
173 -0.261710 -0.178725 -0.005567  
174  0.152804 -0.060837  0.065179  
175 -0.037146 -0.153829  0.234048  
176  0.435575 -0.433720  0.035253  
177 -0.080964 -0.008366 -0.034391  

[178 rows x 10 columns]

In [24]:
#Creating Y_test dataframe after SMOTE
Y_test = pd.DataFrame(Y_test, columns=['bug'])
Y_test

bug
0      0
1      0
2      0
3      0
4      0
..   ...
173    0
174    0
175    0
176    1
177    0

[178 rows x 1 columns]

In [25]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,components

In [26]:
# Preparing the data for the model.
X_train_matrix = X_train.values
X_test_matrix = X_test.values
Y_train_matrix = Y_train.values
Y_test_matrix = Y_test.values

In [27]:
# Preparing the data for the model.
Ytrainseries = Y_train['bug']
Ytestseries = Y_test['bug']

In [28]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [29]:
#Building the model
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=1, activation='relu',input_shape=input_shape))
model.add(Conv2D(32, kernel_size=1, activation='relu'))
model.add(Conv2D(16, kernel_size=1, activation='relu'))
    
    
#model.add(MaxPool2D(pool_size=(1,8)))
#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

#compile model using mse as the loss function
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1, 10, 64)         128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 10, 32)         2080      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 10, 16)         528       
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 1288      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 4,033
Trainable params: 4,033
Non-trainable params: 0
______________________________________________________

In [31]:
# Fitting the model on training data.
history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 128)

Epoch 1/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5265 - mse: 0.5265 - mae: 0.2761 - root_mean_squared_error: 0.7256 - mean_squared_logarithmic_error: 0.1470
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5212 - mse: 0.5212 - mae: 0.2843 - root_mean_squared_error: 0.7220 - mean_squared_logarithmic_error: 0.1444
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5157 - mse: 0.5157 - mae: 0.2936 - root_mean_squared_error: 0.7181 - mean_squared_logarithmic_error: 0.1419
Epoch 4/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5093 - mse: 0.5093 - mae: 0.3041 - root_mean_squared_error: 0.7136 - mean_squared_logarithmic_error: 0.1391
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5025 - mse: 0.5025 - mae: 0.3161 - root_mean_squared_error: 0.7089 - mean_squared_logarithmic_error: 0.1365
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.4953 - mse: 0.4953 - m

1/1 [==============================] - 0s 2ms/step - loss: 0.3790 - mse: 0.3790 - mae: 0.3744 - root_mean_squared_error: 0.6156 - mean_squared_logarithmic_error: 0.1055
Epoch 47/100
1/1 [==============================] - 0s 2ms/step - loss: 0.3755 - mse: 0.3755 - mae: 0.3687 - root_mean_squared_error: 0.6128 - mean_squared_logarithmic_error: 0.1041
Epoch 48/100
1/1 [==============================] - 0s 3ms/step - loss: 0.3720 - mse: 0.3720 - mae: 0.3625 - root_mean_squared_error: 0.6099 - mean_squared_logarithmic_error: 0.1027
Epoch 49/100
1/1 [==============================] - 0s 3ms/step - loss: 0.3685 - mse: 0.3685 - mae: 0.3567 - root_mean_squared_error: 0.6071 - mean_squared_logarithmic_error: 0.1014
Epoch 50/100
1/1 [==============================] - 0s 3ms/step - loss: 0.3651 - mse: 0.3651 - mae: 0.3510 - root_mean_squared_error: 0.6042 - mean_squared_logarithmic_error: 0.1002
Epoch 51/100
1/1 [==============================] - 0s 2ms/step - loss: 0.3618 - mse: 0.3618 - mae: 0.3

1/1 [==============================] - 0s 2ms/step - loss: 0.2615 - mse: 0.2615 - mae: 0.2598 - root_mean_squared_error: 0.5113 - mean_squared_logarithmic_error: 0.0739
Epoch 92/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2595 - mse: 0.2595 - mae: 0.2586 - root_mean_squared_error: 0.5094 - mean_squared_logarithmic_error: 0.0735
Epoch 93/100
1/1 [==============================] - 0s 3ms/step - loss: 0.2575 - mse: 0.2575 - mae: 0.2574 - root_mean_squared_error: 0.5075 - mean_squared_logarithmic_error: 0.0730
Epoch 94/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2555 - mse: 0.2555 - mae: 0.2561 - root_mean_squared_error: 0.5054 - mean_squared_logarithmic_error: 0.0725
Epoch 95/100
1/1 [==============================] - 0s 2ms/step - loss: 0.2534 - mse: 0.2534 - mae: 0.2545 - root_mean_squared_error: 0.5034 - mean_squared_logarithmic_error: 0.0718
Epoch 96/100
1/1 [==============================] - 0s 3ms/step - loss: 0.2512 - mse: 0.2512 - mae: 0.2

In [32]:
# Getting the predictions.
predictions_y = model.predict(X_test1)
predictions_y

array([[0.18527845],
       [0.75844914],
       [0.        ],
       [0.37227455],
       [0.5784121 ],
       [0.        ],
       [0.4556652 ],
       [0.        ],
       [0.        ],
       [0.35433486],
       [0.17013136],
       [1.1797955 ],
       [0.        ],
       [0.53410167],
       [0.73747677],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.4925614 ],
       [2.2172089 ],
       [0.2630758 ],
       [0.21275052],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.53379   ],
       [0.        ],
       [0.        ],
       [0.40744382],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.92756164],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.00860091],
       [0.516

In [33]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],

In [34]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1)

In [35]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([41.], dtype=float32)

In [36]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

array([0.47341725], dtype=float32)

In [37]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

array([0.48184437], dtype=float32)

In [38]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(X_test1, Y_test_matrix)
print(history.history.keys())
model.test_on_batch(X_test1, Y_test_matrix)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 2ms/step - loss: 0.4125 - mse: 0.4125 - mae: 0.3724 - root_mean_squared_error: 0.6422 - mean_squared_logarithmic_error: 0.1559
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [39]:
# Saving the model
model_id = 27
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model2/ant1.3_ant1.4_model2_6'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model2/ant1.3_ant1.4_model2_6/assets


In [40]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [41]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [42]:
score

[27,
 'ant-1.3',
 'ant-1.4',
 0.41245728731155396,
 0.41245728731155396,
 0.3723917007446289,
 0.642228364944458,
 0.15588009357452393,
 0.4734172523021698,
 0.48184436559677124]

In [43]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [44]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)